# #1 Clueless Sudoku - Formulation
_Author: Aster Santana_  
_June, 2021_

This is the MIP formulation of the puzzle. Statement and solution implementation of all puzzles 
are available from the main page of the [Fun Puzzles](https://mip-master.github.io/puzzles/) project, 
which is maintained by [Mip Master](https://mipmaster.org/).

The goal is to fill digits into a grid in way that no digit repeats in any row, column, or pre-defined block, 
with the additional requirement that the sum of the digits in every block is the same.

Although it's possible to do some math and derive the sum that we should have in every block, 
we will let the solver to figure that out for us.

### Input Data
We start by defining three set of indices (which turn out to be the same but we keep them separate for clarity):
- Set of rows:  
    `I = {1, 2, 3, 4, 5, 6}`
- Set of columns:  
    `J = {1, 2, 3, 4, 5, 6}`  
- Set of digits:  
    `K = {1, 2, 3, 4, 5, 6}`

We also need the definition of the blocks (given in the statement of the puzzle):  
`B = {
    1: [(1, 1), (1, 2), (1, 3), (2, 1)], 
    2: [(1, 4), (1, 5), (2, 5)], 
    3: [(1, 6), (2, 6)],
    4: [(2, 2), (2, 3)], 
    5: [(2, 4), (3, 4)], 
    6: [(3, 1), (4, 1)], 
    7: [(3, 2), (3, 3), (4, 3)],
    8: [(3, 5), (3, 6), (4, 5)], 
    9: [(5, 1), (5, 2), (4, 2)], 
    10: [(4, 4), (5, 4)],
    11: [(5, 5), (5, 6), (4, 6)],
    12: [(6, 1), (6, 2)], 
    13: [(5, 3), (6, 3)], 
    14: [(6, 4), (6, 5), (6, 6)]}`
    
### Decision Variables
There are two decisions to be made: What digit goes in each cell and what is the sum of the digits in every block of cells?

So we define one set of binary variables and one continuous variable:
- $x_{ijk}$ equals $1$ if digit $k$ goes in cell $(i, j)$, $0$ otherwise.
- $z$ is the sum of the digits in every block.

### Constraints
Four constraints are required to model this puzzle.
- *Digits can't repeat in any row*:
$$
\sum_j x_{ijk} = 1, \quad \forall i, k.
$$
These constraints say that, given a row $i$ and a digit $k$, if we sum the $x$ variables across the columns, the result has to be $1$, i.e., only one variable takes value $1$.  
If you are not familiar with summation notation, it may be helpful to look at an example. For instance, let's look at $i=2$ and $k=5$. In this case we have $x_{215}+x_{225}+x_{235}+x_{245}+x_{255}+x_{265}=1$, which means that one of the variables in this expression takes value $1$ and all the others take value $0$.
- *Digits can't repeat in any column*:
$$
\sum_i x_{ijk} = 1, \quad \forall j, k.
$$
- *Exactly one digit is assigned to each cell*:
$$
\sum_k x_{ijk} = 1, \quad \forall i, j.
$$
- *The sum of digits in every block must be $z$*:
$$
\sum_{(i, j) \in b}\sum_{k\in K} kx_{ijk} = z, \quad \forall b\in B.
$$
The trick here is to multiply $x_{ijk}$ by $k$. For most cases, $x_{ijk}$ will be zero because of previous constraints. Only when $x_{ijk} = 1$ is that $k$ gets added. 

### Objective Function
There is no objective to maximize or minimize in this problem. We only need to find one infeasible solution (which turns out to be unique in this case). But there is no problem if we define an objective function like this:
$$\min z.$$

### Final Formulation
$$
\begin{align*}
\min & \; z\\
\text{s.t.} & \sum_j x_{ijk} = 1, \quad \forall i, k,\\
& \sum_i x_{ijk} = 1, \quad \forall j, k,\\
& \sum_k x_{ijk} = 1, \quad \forall i, j, \\
& \sum_{(i, j) \in b}\sum_{k\in K} kx_{ijk} = z, \quad \forall b\in B,\\
& x_{ijk} \in \{0, 1\}, \quad \forall i, j, k,\\
& z \geq 0.
\end{align*}
$$